In [ ]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [ ]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView("home_sales")

# Verify that the view is created by running a simple query on it

spark.sql("SELECT * FROM home_sales LIMIT 5").show()

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

query = """
    SELECT 
        YEAR(date_sold) AS year, 
        ROUND(AVG(price), 2) AS avg_price
    FROM 
        home_sales
    WHERE 
        bedrooms = 4
    GROUP BY 
        YEAR(date_sold)
    ORDER BY 
        year
"""

# Run the query
result = spark.sql(query)

# Show the results
result.show()



In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

query = """
    SELECT 
        YEAR(date_built) AS year_built, 
        ROUND(AVG(price), 2) AS avg_price
    FROM 
        home_sales
    WHERE 
        bedrooms = 3 AND bathrooms = 3
    GROUP BY 
        YEAR(date_built)
    ORDER BY 
        year_built
"""

# Run the query
result = spark.sql(query)

# Show the results
result.show()

In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

query = """
    SELECT 
        YEAR(date_built) AS year_built, 
        ROUND(AVG(price), 2) AS avg_price
    FROM 
        home_sales
    WHERE 
        bedrooms = 3 
        AND bathrooms = 3 
        AND floors = 2 
        AND sqft_living >= 2000
    GROUP BY 
        YEAR(date_built)
    ORDER BY 
        year_built
"""

# Run the query
result = spark.sql(query)

# Show the results
result.show()

In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """
    SELECT 
        view, 
        ROUND(AVG(price), 2) AS avg_price
    FROM 
        home_sales
    GROUP BY 
        view
    HAVING 
        AVG(price) >= 350000
    ORDER BY 
        view DESC
"""

# Run the query
result = spark.sql(query)

# Show the results
result.show()

# Calculate the time taken to run the query
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 7. Cache the the temporary table home_sales.

spark.catalog.cacheTable("home_sales")

# Verify that the table is cached
if spark.catalog.isCached("home_sales"):
    print("The home_sales table is cached.")
else:
    print("The home_sales table is not cached.")


In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

query = """
SELECT view_rating, 
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE price >= 350000
GROUP BY view_rating
ORDER BY view_rating DESC
"""

# Run the query on the cached home_sales table
cached_result = spark.sql(query)

# Show the result
cached_result.show()

# Calculate the runtime for the cached query
print("--- Cached Query Run Time: %s seconds ---" % (time.time() - start_time))

In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 

home_sales.write.partitionBy("date_built").parquet("path_to_save/home_sales_partitioned.parquet")

In [ ]:
# 11. Read the formatted parquet data.

partitioned_home_sales = spark.read.parquet("path_to_save/home_sales_partitioned.parquet")

In [ ]:
# 12. Create a temporary table for the parquet data.

partitioned_home_sales.createOrReplaceTempView("parquet_home_sales")

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

result_parquet = spark.sql("""
    SELECT view_rating, 
           ROUND(AVG(price), 2) AS avg_price
    FROM parquet_home_sales
    WHERE price >= 350000
    GROUP BY view_rating
    ORDER BY view_rating DESC
""")

# Show the result
result_parquet.show()

# Calculate and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached

is_cached = spark.catalog.isCached("home_sales")

# Print the result
if is_cached:
    print("The 'home_sales' table is cached.")
else:
    print("The 'home_sales' table is NOT cached.")